In [28]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import json
import riskfolio
from riskfolio.AuxFunctions import weights_discretizetion

from trading import Asset
from dateutil.relativedelta import relativedelta


In [42]:
end = date(2018,12,1) - timedelta(days = 1)
start = end - relativedelta(months = 26)
frequency = "1m"
broker = "gbm"
fiat = "mx"
from_ = "db"
valor_portafolio = 10000

In [43]:
returns_file = "2018-12-01 00-00-00_2019-01-01 00-00-00_analysis.json"
returns_pwd = r"C:\Users\ramon\Documents\Trading\WorkingDir\gbm\results\mx\1m_1\LR_DSFilterDown\72_\{}".format(returns_file)

In [44]:
with open( returns_pwd, "r" ) as fp:
    returns = json.load( fp )

In [45]:
returns = pd.DataFrame.from_dict( returns, orient="index")

In [57]:
df = pd.DataFrame()
for i in returns.index.to_list()[0:20]:

    inst = Asset(
        i,
        start = start,
        end = end,
        frequency=frequency,
        broker = broker,
        fiat = fiat,
        from_ = from_
    )

    if inst.df is None or len(inst.df) == 0: continue

    df = pd.concat([ df, inst.df["close"] ], axis = 1)
    df.rename(columns = {"close":i}, inplace = True)

In [58]:
nans = df.isna().sum(axis = 1)
nans = nans[ nans == len(df.columns) ].index.to_list()
df.drop(df.loc[nans].index, inplace = True)

nans = df.isna().sum()
nans = nans[ nans > int( len(df) * 0.2 ) ].index.to_list()
df.drop( columns = nans, inplace = True )

In [59]:
df.head()

,AAPL,ABBV,AC,AEROMEX,AGUA,ALPEKA,ALSEA,ARA,AXTELCPO,BABAN,BAC,BACHOCOB,BAFARB,BBVA,BIL,BIMBOA,BRKB,CEMEXCPO,CERAMICB,CIEB
2016-10-01,538.434998,1062.020020,117.620003,37.320000,29.780001,28.139999,70.540001,7.12,4.69,1932.760010,312.839996,83.699997,33.798161,135.559998,91.458054,50.910000,NaN,15.730769,44.5,18.0
2016-11-01,570.234985,1246.930054,108.559998,35.970001,27.350000,28.309999,60.400002,6.91,4.07,1939.280029,434.329987,80.599998,33.040180,125.959999,91.462044,47.349998,NaN,15.394230,42.5,15.5
2016-12-01,598.695007,1297.699951,108.070000,39.099998,26.629999,24.770000,59.330002,6.49,3.52,1816.000000,457.019989,84.750000,35.469604,139.000000,91.420113,47.009998,NaN,15.903846,46.5,16.5
2017-01-01,631.974976,1254.920044,112.500000,38.980000,25.590000,23.990000,60.290001,6.68,3.70,2123.120117,473.730011,80.980003,35.469604,142.800003,1898.380005,46.230000,3450.0,18.519230,47.0,15.5
2017-02-01,686.757507,1222.719971,112.510002,44.689999,23.620001,21.320000,56.650002,6.05,3.68,2062.149902,495.000000,80.080002,35.469604,130.339996,1833.959961,47.419998,3428.0,16.355768,47.0,16.0


In [60]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [61]:
latest_price = df.iloc[-1]

port = riskfolio.Portfolio(returns = df.pct_change(periods = 1).dropna())

In [62]:
port.assets_stats()

In [64]:
port.mu = returns[0][0:20]

In [65]:
port.mu

AAPL       -0.300599
ABBV       -0.163134
AC          0.758766
AEROMEX    -0.032128
AGUA       -0.127144
ALPEKA     -0.036463
ALSEA       0.072362
ARA         0.040062
AXTELCPO   -0.039275
BABAN      -0.086491
BAC         0.614308
BACHOCOB   -0.053347
BAFARB     -0.002660
BBVA       -0.220047
BIL         0.005471
BIMBOA      0.040998
BRKB        0.068166
CEMEXCPO    0.032544
CERAMICB   -0.422376
CIEB       -0.102429
Name: 0, dtype: float64

In [67]:
model = 'Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = "MV"        
obj = "Sharpe"

hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate

l = 2 # Risk aversion factor, only useful when obj is 'Utility'
        # Es el factor de cuanto un inversionista es capaz de aceptar riesgo

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

discrete = weights_discretizetion(w, latest_price, valor_portafolio)

discrete = discrete[0].to_dict()

total_money = 0
for i in discrete:
    total_money += ( latest_price[i]*discrete[i] )

discrete_weights = {}
for i in discrete:
    discrete_weights[i] = (latest_price[i]*discrete[i]) / total_money

In [68]:
discrete_weights

{'AAPL': 0.0,
 'ABBV': 0.0,
 'AC': 0.5792013290176143,
 'AEROMEX': 0.013033720091727022,
 'AGUA': 0.0,
 'ALPEKA': 0.0,
 'ALSEA': 0.0,
 'ARA': 0.0,
 'AXTELCPO': 0.0,
 'BABAN': 0.0,
 'BAC': 0.4077649508906586,
 'BACHOCOB': 0.0,
 'BAFARB': 0.0,
 'BBVA': 0.0,
 'BIL': 0.0,
 'BIMBOA': 0.0,
 'BRKB': 0.0,
 'CEMEXCPO': 0.0,
 'CERAMICB': 0.0,
 'CIEB': 0.0}

: 